### GENERATE_DH

In [15]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives import serialization

class DH_pair:
    def __init__(self, dh_priv, dh_pub) -> None:
        self.dh_priv = dh_priv
        self.dh_pub = dh_pub

def GENERATE_DH():
    # Generate a new DH pair key.
    dh_priv = X25519PrivateKey.generate()
    dh_pub = dh_priv.public_key()
    
    return DH_pair(dh_priv, dh_pub)

dh_pair = GENERATE_DH()
dh_pair2 = GENERATE_DH()

print("Private key:", dh_pair.dh_priv.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption()))
print("Length Private key:", len(dh_pair.dh_priv.private_bytes(encoding=serialization.Encoding.Raw, format=serialization.PrivateFormat.Raw, encryption_algorithm=serialization.NoEncryption())))
print()
print("Public key:", dh_pair.dh_pub.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw))
print("Length Public key:", len(dh_pair.dh_pub.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)))


Private key: b"\xc8\x83\xc4\xaa\x96\xa3(\xccm\x8c\x10\x85\xb9\x1b4d\x0b\xb9\xfe'\nsAK\x8e(5\xdb\xa0\x99\x81b"
Length Private key: 32

Public key: b'\x8b\xc7\x93\xff\x04\xfa.\xc8\xa3?\xb6\xc0}\\\x87\xd0,DKW\x9d\x9b /;\x1c\xd7\x832\x1dfK'
Length Public key: 32


### DH

In [11]:
def DH(dh_pair: DH_pair, dh_pub):
    # Perform DH calculation between private key and public key
    return dh_pair.dh_priv.exchange(dh_pub)

# Calcular la clave compartida usando la función DH()
dh_out = DH(dh_pair=dh_pair, dh_pub=dh_pair2.dh_pub)
dh_out2 = DH(dh_pair=dh_pair2, dh_pub=dh_pair.dh_pub)
print("Shared secret 1:", dh_out)
print("Shared secret 2:", dh_out2)

Shared secret 1: b'y\xec\xec\x0b\xf4\xea\xc1\xbc\xa0\x12[\x94eJ\x8a~\xc3\x92\xa5\xf5\xe9\xfd\xff\xc6\xb72\xc1K\xbbk\x9b\x0e'
Shared secret 2: b'y\xec\xec\x0b\xf4\xea\xc1\xbc\xa0\x12[\x94eJ\x8a~\xc3\x92\xa5\xf5\xe9\xfd\xff\xc6\xb72\xc1K\xbbk\x9b\x0e'


### KDF_RK

In [12]:
import secrets

rk = secrets.token_bytes(32)
print("Random",len(rk),"bytes:", rk)

Random 32 bytes: b'\\\x0e\xef\x9f\x05\x85\xe8H\xd6&\xb8\x93\xe0\xeb\x1awP\xe5\x8c\x08\x86\xa3\xbf\x99\xa4\xfe\xfa\x14\x13"\x88\xfc'


In [13]:
from hkdf import hkdf_expand, hkdf_extract
from binascii import unhexlify
import hashlib

def KDF_RK(rk: bytes, dh_out: bytes) -> [[bytes]*32, [bytes]*32]:
    prk = hkdf_extract(salt=rk, input_key_material=dh_out, hash=hashlib.sha512)
    key = hkdf_expand(pseudo_random_key=prk, info=b"Info KDF_RK", length=64, hash=hashlib.sha512)

    return key[:32],key[32:] # Root Key, Chain Key

rk, ck = KDF_RK(rk=rk, dh_out=dh_out)
print("Root Key:", rk)
print("Length Root Key:", len(rk), "\n")
print("Chain Key:", ck)
print("Length Chain Key:", len(ck))

Root Key: b'\x84\xb0\xf3W\x9bi{?e\xb5\xf5\x1dV<\xa2\xcfD\x98(\xdc-\xec\\\x82\xf2\x1d\x18+\x17\x9f\x11\xa0'
Length Root Key: 32 

Chain Key: b'/\xa1-\xb3\\\xbd(\xee\xa7\xd1\x0c\x0eW\xdf\x80\xf3\xf8,\xd9\x0f\xa6\xeaC\xa2~d},l8.\xa8'
Length Chain Key: 32


### KDF_CK

In [14]:
import hmac

def KDF_CK(ck: bytes) -> [[bytes]*32, [bytes]*32]:
    new_ck = hmac.new(key=ck, msg=b'Chain Key', digestmod=hashlib.sha512)
    mk = hmac.new(key=ck, msg=b'Message Key', digestmod=hashlib.sha512)

    return new_ck.digest()[:32], mk.digest()[:32] # Chain Key, Message Key


ck, mk = KDF_CK(ck=ck)

print("Chain Key:",ck)
print("Length Chain Key:",len(ck))
print()
print("Message Key:",mk)
print("Length Message Key:",len(mk))


Chain Key: b'\x96z\x15!\x1a\xc2i\x16\x91\xc9n\x0c\xf5q)\x88\xeej\xd9\xe9\xe3\x01\xda\xd12\xdf;\xf2\xbc\x02\xba\xf1'
Length Chain Key: 32

Message Key: b'\xcd\xbc\xe4\xf8\xaf\xc2\x04\x9f\x7f\x06AL\xb2"\xef\xf3H\xdb^\xa9"\x8c\x96\x93\xb0h\x08W*c?"'
Length Message Key: 32


### ENCRYPT

In [77]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding  # - Import to use the padding.
from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

def ENCRYPT(mk: bytes, plaintext: bytes, assotiate_data: bytes):
    # Key Derivation Function
    output_length = 80
    prk = hkdf_extract(salt=bytes(output_length), input_key_material=mk, hash=hashlib.sha512)
    exp = hkdf_expand(pseudo_random_key=prk, info=b"Info ENCRYPT", length=output_length, hash=hashlib.sha512)

    ek = exp[:32] # Encryption Key
    ak = exp[32:64] # Authentication Key
    iv = exp[64:] # Initialization vector

    # Construct an AES-128-GCM Cipher object with the generated encryption key and IV.
    encryptor = Cipher(
            algorithms.AES(key=ek),
            modes.GCM(initialization_vector=iv),
            backend=default_backend()
        ).encryptor()

    padder = padding.PKCS7(128).padder()
    padder_data = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padder_data) + encryptor.finalize()
    tag = encryptor.tag # Tag is always 16B long.

    # HMAC for authentication of the associate it data.
    ad = hmac.new(key=ak, msg=assotiate_data, digestmod=hashlib.sha512).digest() # 512b = 64B
    print(len(ad))
    
    return tag + ad + ciphertext

enc_msg = ENCRYPT(mk=mk,plaintext=b"hola",assotiate_data=b"hola")

64


### HEADER

In [19]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PublicKey

class HEADER:
    def __init__(self, dh_pair=None, pn: int = 0, n: int = 0) -> None:
        if dh_pair is not None:
            self.dh = dh_pair.dh_pub
            self.pn = pn
            self.n = n

    def to_bytes(self):
        dh_pub_b = self.dh.public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw) # 32 B
        pn_b = self.pn.to_bytes(1, byteorder='big') # 1 B
        n_b = self.n.to_bytes(1, byteorder='big') # 1 B

        return dh_pub_b + pn_b + n_b 
    
    def to_object(self, header_bytes: [bytes]*34):
        self.dh = X25519PublicKey.from_public_bytes(data=header_bytes[:32])
        self.pn = int.from_bytes(header_bytes[32:33], byteorder='big')
        self.n = int.from_bytes(header_bytes[33:34], byteorder='big')

header = HEADER(dh_pair=dh_pair, pn=4, n=3)
print(header)

In [101]:


dh_pub = X25519PublicKey.from_public_bytes(data=header[:32])

### CONCAT

In [23]:
def CONCAT(ad: bytes, header: HEADER):
    return ad + header.to_bytes()

concat = CONCAT(ad=b"hola", header=header)
print(concat)
h2 = HEADER()
h2 = h2.to_object(header_bytes=concat())

b'hola\x8b\xc7\x93\xff\x04\xfa.\xc8\xa3?\xb6\xc0}\\\x87\xd0,DKW\x9d\x9b /;\x1c\xd7\x832\x1dfK\x04\x03'


### MESSAGE TO SEND

In [ ]:
header = HEADER

### DECRYPT

In [18]:
a = 2
a = a.to_bytes(1, byteorder='big')
print(a)
print(type(a))
a = int.from_bytes(a, byteorder='big')
print(a)
print(type(a))

b'\x02'
<class 'bytes'>
2
<class 'int'>


In [26]:
len(hashlib.sha256(b'hola').digest())

32